In [1]:
import numpy as np
from scipy import fft
import xarray as xr
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def plotPSDandAutoCorr(psd_WSPD, 
        psd_cosWDIR, 
        psd_sinWDIR, 
        corrWSPD, 
        corrCosWDIR, 
        corrSinWDIR, 
        title, fileName):
    
    freq = fft.fftshift(fft.fftfreq(5000, d=10))
    mask = freq>0
    freq = freq[mask]
    
    halfNdata = int(5000//2)
    xdata = np.arange(0, halfNdata)*10/60
    
    
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(21,10))
        
    ax= axes[0, 0]
    median = np.median(psd_WSPD, axis=0)
    q25 = np.quantile(psd_WSPD, 0.25, axis=0)
    q75 = np.quantile(psd_WSPD, 0.75, axis=0)
    ax.plot(freq*60, median)
    ax.fill_between(freq*60, q25, q75, alpha = 0.25)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel(r'Wind Speed PSD $[m^2/cycle]$')
    ax.set_xlabel(r'wave number (per hour)')
    
    ax= axes[0, 1]
    median = np.median(psd_cosWDIR, axis=0)
    q25 = np.quantile(psd_cosWDIR, 0.25, axis=0)
    q75 = np.quantile(psd_cosWDIR, 0.75, axis=0)
    ax.plot(freq*60, median)
    ax.fill_between(freq*60, q25, q75, alpha = 0.25)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel(r'cosine Wind Direction PSD $[degrees^2/cycle]$')
    ax.set_xlabel(r'wave number (per hour)')
    
    ax= axes[0, 2]
    median = np.median(psd_sinWDIR, axis=0)
    q25 = np.quantile(psd_sinWDIR, 0.25, axis=0)
    q75 = np.quantile(psd_sinWDIR, 0.75, axis=0)
    ax.plot(freq*60, median)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.fill_between(freq*60, q25, q75, alpha = 0.25)
    ax.set_ylabel(r'sine Wind Direction PSD $[degrees^2/cycle]$')
    ax.set_xlabel(r'wave number (per hour)')
    
    
    ax= axes[1, 0]
    s = 0
    e = 1000
    mean = np.mean(corrWSPD, axis=0)
    std = np.std(corrWSPD, axis=0)
    ax.plot(xdata[s:e], mean[s:e])
    ax.fill_between(xdata[s:e], mean[s:e]- std[s:e], mean[s:e] + std[s:e], alpha=0.25)
    ax.set_ylabel(r'autocorrelation Wind Speed')
    ax.set_xlabel(r'Time lag (hour)')
    ax.grid(True)
    
    ax= axes[1, 1]
    mean = np.mean(corrCosWDIR, axis=0)
    std = np.std(corrCosWDIR, axis=0)
    ax.plot(xdata[s:e], mean[s:e])
    ax.fill_between(xdata[s:e], mean[s:e]- std[s:e], mean[s:e] + std[s:e], alpha=0.25)
    ax.set_ylabel(r'autocorrelation cosine Wind Direction')
    ax.set_xlabel(r'Time lag (hour)')
    ax.grid(True)
    
    ax= axes[1, 2]
    mean = np.mean(corrSinWDIR, axis=0)
    std = np.std(corrSinWDIR, axis=0)
    ax.plot(xdata[s:e], mean[s:e])
    ax.fill_between(xdata[s:e], mean[s:e]- std[s:e], mean[s:e] + std[s:e], alpha=0.25)
    ax.set_ylabel(r'autocorrelation sine Wind Direction')
    ax.set_xlabel(r'Time lag (hour)')
    ax.grid(True)

    plt.suptitle(title)
    plt.savefig(fileName, dpi = 100)
    plt.close()

In [3]:
def getFFT_autoCorr(bigDS, start, end):
    ds = bigDS.isel(TIME=slice(start, end))
    
    WSPD = ds['WSPD_10N'].sel(HEIGHT=10).to_numpy()
    WDIR = ds['WDIR'].sel(HEIGHT=4).to_numpy()
    WDIR = (WDIR+360)%360

    WSPD = WSPD-np.mean(WSPD)
    
    
    cosWDIR = np.cos(np.deg2rad(WDIR))
    sinWDIR = np.sin(np.deg2rad(WDIR))

    cosWDIR = cosWDIR-np.mean(cosWDIR)
    sinWDIR = sinWDIR-np.mean(sinWDIR)
    
    WSPD_hann = np.hanning(len(WSPD)) * WSPD
    cosWDIR_hann = np.hanning(len(cosWDIR)) * cosWDIR
    sinWDIR_hann = np.hanning(len(sinWDIR)) * sinWDIR
    
    WSPD_hat = fft.fftshift(fft.fft(WSPD_hann, norm='forward'))
    cosWDIR_hat = fft.fftshift(fft.fft(cosWDIR_hann, norm='forward'))
    sinWDIR_hat = fft.fftshift(fft.fft(sinWDIR_hann, norm='forward'))
    freq = fft.fftshift(fft.fftfreq(len(WSPD), d=10))
    
    mask = freq>0
    WSPD_hat = WSPD_hat[mask]
    cosWDIR_hat = cosWDIR_hat[mask]
    sinWDIR_hat = sinWDIR_hat[mask]
    freq = freq[mask]
        
    WSPD = ds['WSPD_10N'].sel(HEIGHT=10).to_numpy()
    WDIR = ds['WDIR'].sel(HEIGHT=4).to_numpy()
    WDIR = (WDIR+360)%360
    
    cosWDIR = np.cos(np.deg2rad(WDIR))
    sinWDIR = np.sin(np.deg2rad(WDIR))
    
    ndata = len(WSPD)
    halfNdata = int(ndata//2)
    xdata = np.arange(0, halfNdata)*10/60
    
    corrWSPD = np.zeros((halfNdata,), dtype=float)
    corrCosWDIR = np.zeros((halfNdata,), dtype=float)
    corrSinWDIR = np.zeros((halfNdata,), dtype=float)
    
    for i in range(halfNdata):
        corrWSPD[i] = np.corrcoef(WSPD[0:halfNdata],WSPD[i:halfNdata+i] )[0,1]
        corrCosWDIR[i] = np.corrcoef(cosWDIR[0:halfNdata],cosWDIR[i:halfNdata+i] )[0,1]
        corrSinWDIR[i] = np.corrcoef(sinWDIR[0:halfNdata],sinWDIR[i:halfNdata+i] )[0,1]

    return WSPD_hat.real**2, cosWDIR_hat.real**2, sinWDIR_hat.real**2, corrWSPD, corrCosWDIR, corrSinWDIR
    

In [4]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0

for task in taskList:
    lat = task[0]
    lon = task[1]
    
    LAT = lat
    LON = lon
    
    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'
    
    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)
    
    bFileName = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_COARE3p5_2000.nc'
    if not os.path.isfile(bFileName):
        print(bFileName, 'not present')
        continue
    ds = xr.open_dataset(bFileName)
    mask1 = ds.sel(HEIGHT=4)['WSPD_QC'].isin([1,2]).to_numpy()
    mask2 = ds.sel(HEIGHT=4)['WDIR_QC'].isin([1,2]).to_numpy()
    mask3 = ds.sel(DEPTH=1)['SST_QC'].isin([1,2]).to_numpy()
    mask4 = ds.sel(HEIGHT=3)['RELH_QC'].isin([1,2]).to_numpy()
    mask5 = ds.sel(HEIGHT=3)['AIRT_QC'].isin([1,2]).to_numpy()
    
    selectMask = np.logical_and(mask1, mask2)
    selectMask = np.logical_and(selectMask, mask3)
    selectMask = np.logical_and(selectMask, mask4)
    selectMask = np.logical_and(selectMask, mask5)
    indices = selectMask.nonzero()[0]
    
    ds = ds.isel(TIME=indices)
    
    time = pd.to_datetime(ds['TIME'].to_numpy())
    deltaTime = np.roll(time, -1) - time
    deltaTime = deltaTime[0:-1]
    deltaTime = np.array(deltaTime, dtype='timedelta64[s]')
    mask = np.logical_or(abs(deltaTime) > np.array([602], dtype='timedelta64[s]') , abs(deltaTime) < np.array([508], dtype='timedelta64[s]'))

    stopIndices = mask.nonzero()[0]+1
    startIndices = np.roll(stopIndices,1)
    startIndices[0] = 0    
    
    diff = stopIndices - startIndices
    diff = diff[0:-1]
    
    startIndexList = []
    endIndexList = []
    
    for i in range(len(diff)):
        startIndex = startIndices[i]
        endIndex = stopIndices[i]
    
        
        subStartIndex = startIndex
        subEndIndex = subStartIndex + 5000
        if diff[i] > 5000:
            while True:
                subEndIndex = subStartIndex + 5000
                if subEndIndex > endIndex:
                    break
                startIndexList.append(subStartIndex)
                endIndexList.append(subEndIndex)
                subStartIndex = subEndIndex
                
    ntseries = len(startIndexList)  
            
    psd_WSPD = np.zeros((ntseries, 2499), dtype=float)
    psd_cosWDIR = np.zeros((ntseries, 2499), dtype=float)
    psd_sinWDIR = np.zeros((ntseries, 2499), dtype=float)
    corrWSPD = np.zeros((ntseries, 2500), dtype=float)
    corrCosWDIR = np.zeros((ntseries, 2500), dtype=float)
    corrSinWDIR = np.zeros((ntseries, 2500), dtype=float)     
    
    for i in range(ntseries):
        this_start = startIndexList[i]
        this_end = endIndexList[i]
        
        psd_WSPD[i,:], psd_cosWDIR[i,:], psd_sinWDIR[i,:], corrWSPD[i,:], corrCosWDIR[i,:], corrSinWDIR[i,:] = getFFT_autoCorr(ds, this_start, this_end)
    
    title = f'{lat:02d}{latUnits} {lon:03d}{lonUnits}'
    fileName = f'SpectralPlotsAveraging/{lat:02d}{latUnits}_{lon:03d}{lonUnits}_PSDandAutoCorr.png'
    plotPSDandAutoCorr(psd_WSPD, 
            psd_cosWDIR, 
            psd_sinWDIR, 
            corrWSPD, 
            corrCosWDIR, 
            corrSinWDIR, 
            title, fileName)
    ds.close()
        
        

../../../downloads/Buoy/extractedGZ/WINDS/T_09S_095W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_110W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_125W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_140W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_155W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_170W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_180W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_09S_165E_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_08S_140W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_08S_180W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_05S_180W_xrr_COARE3p5_2000.nc not present
../../../downloads/Buoy/extractedGZ/WINDS/T_05N_180W_x

In [ ]:
lat = -5
lon = -140

LAT = lat
LON = lon

if lat < 0:
    latUnits = 'S'
else:
    latUnits = 'N'

if lon < 0:
    lonUnits = 'W'
else:
    lonUnits = 'E'

LON = (LON+360)%360
lat=abs(lat)
lon=abs(lon)

bFileName = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_COARE3p5_2000.nc'
# if not os.path.isfile(bFileName):
#     print(bFileName, 'not present')
#     continue
ds = xr.open_dataset(bFileName)
mask1 = ds.sel(HEIGHT=4)['WSPD_QC'].isin([1,2]).to_numpy()
mask2 = ds.sel(HEIGHT=4)['WDIR_QC'].isin([1,2]).to_numpy()
mask3 = ds.sel(DEPTH=1)['SST_QC'].isin([1,2]).to_numpy()
mask4 = ds.sel(HEIGHT=3)['RELH_QC'].isin([1,2]).to_numpy()
mask5 = ds.sel(HEIGHT=3)['AIRT_QC'].isin([1,2]).to_numpy()

selectMask = np.logical_and(mask1, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)
indices = selectMask.nonzero()[0]

ds = ds.isel(TIME=indices)

time = pd.to_datetime(ds['TIME'].to_numpy())
deltaTime = np.roll(time, -1) - time
deltaTime = deltaTime
deltaTime = np.array(deltaTime, dtype='timedelta64[s]')
mask = np.logical_or(abs(deltaTime) > np.array([602], dtype='timedelta64[s]') , abs(deltaTime) < np.array([508], dtype='timedelta64[s]'))
stopIndices = mask.nonzero()[0]+1
startIndices = np.roll(stopIndices,1)
startIndices[0] = 0    

diff = stopIndices - startIndices
diff = diff[0:-1]

startIndexList = []
endIndexList = []

for i in range(len(diff)):
    startIndex = startIndices[i]
    endIndex = stopIndices[i]

    
    subStartIndex = startIndex
    subEndIndex = subStartIndex + 5000
    if diff[i] > 5000:
        while True:
            subEndIndex = subStartIndex + 5000
            if subEndIndex > endIndex:
                break
            startIndexList.append(subStartIndex)
            endIndexList.append(subEndIndex)
            subStartIndex = subEndIndex
            
ntseries = len(startIndexList)  

i=0

this_start = startIndexList[i]
this_end = endIndexList[i]

#testds = ds.isel(TIME=slice(this_start, this_end))

In [ ]:
startIndices

In [ ]:
stopIndices

In [ ]:
np.array(endIndexList) - np.array(startIndexList)

In [ ]:
endIndexList

In [ ]:
startIndexList[-1], endIndexList[-1]


In [ ]:
plt.plot(testds['WSPD_10N'].to_numpy())

In [ ]:
timeArr = ds.TIME.to_numpy()

In [ ]:
timeArr[64005], timeArr[64006]

In [ ]:
ds

In [ ]:
testds = ds.isel(TIME=slice(0, 10))